First working Q table for 1 qubit

In [5]:
from numpy import *
import cmath
from random import randrange
import pandas as pd
import time
from IPython.display import clear_output

Define states and gates

In [21]:
a = 0.5
b = 0.5
one = array([[int(0)], [int(1)]])
zero = array([[int(1)], [int(0)]])
psi1 = (1/(sqrt(2)))*(zero + one)
psi1

array([[0.70710678],
       [0.70710678]])

In [13]:
X = array([[0 , 1], [1, 0]])
Z = array([[1,0],[0,-1]])
Iden = array([[1 , 0], [0, 1]])
Y = array([[0 , -1j], [1j, 0]])
H = (1/(sqrt(2)))*array([[1 , 1], [1, -1]])
S = array([[1 , 0], [0, 1j]])


Define target

In [163]:
right = Y.dot(Z.dot(X.dot(zero)))
right


array([[0.+1.j],
       [0.+0.j]])

Compile actions and starting states into iterable lists

In [25]:
list2 = [Iden, X, Y, Z, H, S]

In [30]:
list1 = [zero, one, psi1]

Defining metric for scoring correct states

In [129]:
def probability(guess, right):
    summ = 0
    for i in range(0,2):
        diff = (guess[i] - right[i])
        summ+= abs(diff)
    score = 1 - (summ)/2
    return(float(score))

Dataframe try

- First set up the dataframe with the correct columns and some possible states

In [144]:
Q_table = pd.DataFrame()

In [145]:
for i in range(0,len(list1)):
    new4 = list1[i]
    for m in range(0,len(list2)):
        gate = m
        new = (list2[m].dot(list1[i]))
        Q = pd.DataFrame({'state':[new4], 'string' : str(new4), 'gate':gate, 'new':str(new) ,'Q':0}, index=[0])
        Q_table = Q_table.append(Q)
#Q_table = Q_table.reset_index(drop=True)

It's easier to call upon cells in string format rather than array format

In [146]:
Q_table = Q_table.astype({'state': str})

In [147]:
Q_tablestr = Q_table.drop(['state'], axis = 1)
Q_tablestr = Q_tablestr.drop_duplicates()
Q_tablestr = Q_tablestr.reset_index(drop=True)

Set variables - these may need to be played about with more to find optimal values

In [164]:
table = Q_tablestr.copy()

In [165]:
alpha = 0.5 #how much we update Q for every new action
gamma = 0.9 #discount factor 

num_repeats = 100


In [166]:

for repeats in range(0,num_repeats):
    k = randrange(len(list1))
    psiold = list1[k]
    reward = 0
    for t in range(0,5): #set the max number of steps taken
        reward += 0.1 #add more penalty with each step
        
        sec = table[table['string']==str(psiold)] #choose the part of the dataframe that contains the current state
        
        gather = pd.DataFrame()
        for n in range(0,(len(sec))): #iterate through the rows that contain the current state
            for z in range(0,len(list2)): # find the action associated with the state in each row
                if(sec.iloc[n]['gate']== z):
                    apply = array(list2[z])
                    zed = z
                    
            psinew = apply.dot(psiold) #create new state by applying action
                
                
            news = table[table['string']==str(psinew)] #choose part of dataframe that contains the new state
            
            if len(news) == 0: #if the new state is not in the dataframe, insert it with all the possible actions
                for s in range(0,len(list2)):
                    lad = list2[s].dot(psinew)
                    news = pd.DataFrame({'string' : str(psinew), 'gate':s, 'new':str(lad) ,'Q':0}, index=[0])
                    table = table.append(news)
                    
            maxQ = max(news['Q']) #find the max Q value the new state
            gather = gather.append(news).drop_duplicates() #create a dataframe of all possible new states
            plus = 0
            if (probability(psiold, right)) == 1: #insert a reward if the new state is the one we want
                plus = 1 
                    
            table.loc[(table.string == str(psiold)) & (table.gate == zed), 'Q'] += alpha*((plus - reward) + gamma*(maxQ) - sec.iloc[n]['Q'])
            #update the Q value of the old state based on the rewards given by the new state
        if plus == 1: #If we have reached the state we want we can stop
            print('goal reached')
            break
        else:
            top = gather.loc[(gather.Q == max(gather['Q']))] #collect the max Q value from all possible new states
            toparray =[] #sort these into an array with their associated action
            actarray =[]
            for o in range(0,len(top)):
                for p in range(0,len(list2)):
                    if (str((array(list2[p])).dot(psiold)) == (top.iloc[o]['string'])):
                        toparray.append((array(list2[p])).dot(psiold))
                        actarray.append(p)
        
            garray =[] #create an array of all possible new states with associated actions
            actg = []
            for a in range(0,len(top)):
                for b in range(0,len(list2)):
                    if (str((array(list2[b])).dot(psiold)) == (gather.iloc[a]['string'])):
                        garray.append((array(list2[b])).dot(psiold))
                        actg.append(b)
        
            q = randrange(100)
            if q > 50: #Create a percentage chance that the next state will be one with a max Q value (expoitation) or a state obtained from a random action (exploration)
                r = randrange(len(toparray))
                psiold = toparray[r]
                action = actarray[r]
            else:
                r = randrange(len(garray))
                psiold = garray[r]
                action = actg[r]
        
        #print(str(t) + 'th state is ' + str(psiold) + ' after action ' + str(action))
    print(repeats)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
goal reached
18
19
goal reached
20
21
22
23
24
25
26
goal reached
27
goal reached
28
29
30
goal reached
31
goal reached
32
33
34
35
goal reached
36
goal reached
37
goal reached
38
goal reached
39
40
41
goal reached
42
43
44
45
goal reached
46
47
48
49
50
goal reached
51
52
goal reached
53
54
goal reached
55
goal reached
56
goal reached
57
58
59
goal reached
60
61
goal reached
62
goal reached
63
goal reached
64
goal reached
65
66
67
68
goal reached
69
70
71
72
73
74
75
goal reached
76
goal reached
77
78
goal reached
79
80
goal reached
81
82
goal reached
83
goal reached
84
85
goal reached
86
87
goal reached
88
89
90
goal reached
91
goal reached
92
goal reached
93
94
goal reached
95
96
goal reached
97
goal reached
98
goal reached
99


In [154]:
table.sort_values(by='Q', ascending =False).head(50)

,string,gate,new,Q
0,[[-0.70710678]\n [ 0.70710678]],0,[[-0.70710678]\n [ 0.70710678]],5.240534
0,[[-0.70710678]\n [ 0.70710678]],1,[[ 0.70710678]\n [-0.70710678]],4.418588
0,[[ 0.70710678]\n [-0.70710678]],1,[[-0.70710678]\n [ 0.70710678]],4.311807
15,[[0.70710678]\n [0.70710678]],3,[[ 0.70710678]\n [-0.70710678]],3.709725
10,[[0]\n [1]],4,[[ 0.70710678]\n [-0.70710678]],3.655715
0,[[ 0.70710678]\n [-0.70710678]],0,[[ 0.70710678]\n [-0.70710678]],3.486685
1,[[1]\n [0]],1,[[0]\n [1]],3.150714
4,[[1]\n [0]],4,[[0.70710678]\n [0.70710678]],3.144737
13,[[0.70710678]\n [0.70710678]],1,[[0.70710678]\n [0.70710678]],3.116721
12,[[0.70710678]\n [0.70710678]],0,[[0.70710678]\n [0.70710678]],3.116721


In [170]:
top1 = pd.DataFrame
for m in range(0,len(list1)):
    
    oppa = table.loc[(table.string == str(list1[m]))]
    top = oppa.loc[(oppa.Q == max(oppa['Q']))]
    for n in range(0,5):
        
        oppa1 = table.loc[(table.string == top['new'].values[n])]
        top1 = oppa1.loc[(oppa1.Q == max(oppa1['Q']))]
        
        top = top.append(top1)
        
    print(top)
    

                  string  gate                    new         Q
2            [[1]\n [0]]     2  [[0.+0.j]\n [0.+1.j]]  3.227715
0  [[0.+0.j]\n [0.+1.j]]     1  [[0.+1.j]\n [0.+0.j]]  4.116595
0  [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0  [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0  [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0  [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
                   string  gate                    new         Q
11            [[0]\n [1]]     5  [[0.+0.j]\n [0.+1.j]]  3.341675
0   [[0.+0.j]\n [0.+1.j]]     1  [[0.+1.j]\n [0.+0.j]]  4.116595
0   [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0   [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0   [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
0   [[0.+1.j]\n [0.+0.j]]     5  [[0.+1.j]\n [0.+0.j]]  4.992557
                           string  gate                    new         Q
16  [[0.70710678]\n [0.7